# LFP example


In [1]:
from bayes_window.generative_models import generate_fake_lfp
from bayes_window.visualization import plot_data, plot_data_slope_trials
from bayes_window import BayesWindow, models

## Make and visualize model oscillation power
40 trials of "theta power" is generated for every animal. It is drawn randomly as a poisson process.

This is repeated for "stimulation" trials, but poisson rate is higher.

Mice vary in their baseline power. Higher-baseline mice tend to have smaller stim response (See plot)

In [2]:
df, df_monster, index_cols, _ = generate_fake_lfp(mouse_response_slope=6, n_trials=30)

In [3]:
c1=plot_data(df=df,x='stim',y='Log power').properties(width=60)
c2=plot_data_slope_trials(df=df,x='stim',y='Log power',color=None,detail='i_trial')
(c1+c2).facet(column='mouse')

alt.FacetChart(...)

In [4]:
plot_data(df=df,x='stim',y='Log power',color='mouse').properties(width=80)

alt.LayerChart(...)

## Fit a Bayesian hierarchical model and plot slopes
In a hierarchical model, parameters are viewed as a sample from a population distribution of parameters. Thus, we view them as being neither entirely different or exactly the same. This is ***partial pooling***:

![hierarchical](../motivation/parpooled.png)
This model allows intercepts to vary across mouse, according to a random effect. We just add a fixed slope for the predictor (i.e all mice will have the same slope):

$$y_i = \alpha_{j[i]} + \beta x_{i} + \epsilon_i$$

where

$$\epsilon_i \sim N(0, \sigma_y^2)$$

and the intercept random effect:

$$\alpha_{j[i]} \sim N(\mu_{\alpha}, \sigma_{\alpha}^2)$$

As with the the no-pooling model, we set a separate intercept for each mouse, but rather than fitting separate regression models for each mouse, multilevel modeling **shares strength** among mice, allowing for more reasonable inference in mice with little data. Here is what that looks in code:

In [17]:
# Initialize:
window=BayesWindow(df, y='Log power', treatment='stim', group='mouse')
# Fit:
window.fit_slopes(add_data=True, model=models.model_hier_stim_one_codition,
                  do_make_change='subtract', dist_y='normal');

# Plot:
chart_power_difference = window.plot(independent_axes=False)
chart_power_difference

alt.LayerChart(...)

In this chart:

- The blue dot is the mean of posterior

- The black line is the 94% highest density interval

- The boxplot is made from difference between groups in the data (no fitting)


## Compare to traditional approaches

ANOVA does not pick up the effect of stim as significant:

In [6]:
window.fit_anova();

Log_power~stim
             sum_sq    df         F    PR(>F)
stim      0.018197   1.0  0.769115  0.395291
Residual  0.331239  14.0       NaN       NaN


A linear mixed-effect model shows the effect of stim (slope) as significant. It includes random intercepts of mouse:

In [ ]:
window.fit_lme()

window.posterior

In [15]:
window.plot_posteriors_no_slope()

alt.LayerChart(...)

## Inspect Bayesian result further
We named intercept _a\_subject_, so now we can plot it for each animal

In [16]:
# Fit slopes again:
window.fit_slopes(add_data=True, model=models.model_hier_stim_one_codition,
                  do_make_change='subtract', dist_y='normal');
intercepts=window.trace.posterior['a_subject'].mean(['chain','draw']).to_dataframe().reset_index()
import altair as alt
chart_intercepts=alt.Chart(intercepts).mark_bar().encode(
    x=alt.X('subject:O',title='Mouse'),
    y=alt.Y('a_subject', title='Intercept')
)
chart_intercepts

alt.Chart(...)

Our backend's flexibility allows us to easily concatenate multiple charts in the same figures with the | operator:

In [9]:
chart_intercepts | chart_power_difference

alt.HConcatChart(...)